# Python Scripting with GRASS

**_Corey White, Caitlin Haedrich and Vaclav Petras_**

_FOSS4G-NA 2025_

Contents:
1. [Set up GRASS environment](#1.-Set-up-GRASS-environment)
2. [Create a GRASS Python function](#2.-Scripting-with-GRASS-Tools)
3. [Convert the function to an executable script](#3.-From-Function-to-Executable-Script)

***

## 1. Set up GRASS environment

Import the Python standard libraries we need.

In [ ]:
import subprocess
import sys

We are going to import the GRASS GIS Python API ([`grass.tools`](https://grass.osgeo.org/grass-devel/manuals/python_intro.html)) and the GRASS GIS Jupyter package ([`grass.jupyter`](https://grass.osgeo.org/grass-devel/manuals/jupyter_intro.html)), but first, we'll need to ask `grass` to check it's `--config` to see where the python packages are then add them to the system path before we can import them. This command is slightly different for each operating system.

We use `subprocess.check_output` to find the path and `sys.path.append` to add it to the path.

In [ ]:
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

And now we can import the GRASS python packages!

In [ ]:
# Import the GRASS GIS packages we need.
# import grass.script as gs
import grass.jupyter as gj
from grass.tools import Tools

Start a GRASS session with `gj.init`. If not using `grass.jupyter` (i.e. writing as script as we will later), start a GRASS session with `grass.script.setup.init()`.

In [ ]:
# gs.run_command(
#     "g.download.project",
#     url="https://grass.osgeo.org/sampledata/north_carolina/nc_spm_full_v2alpha2.tar.gz",
#     path=Path(".")
# )
!grass --tmp-project XY --exec g.download.project url=https://grass.osgeo.org/sampledata/north_carolina/nc_spm_full_v2alpha2.tar.gz path=$HOME

In [ ]:
# gs.create_project("/content/drive/MyDrive/grassdata/nc_sentinel", epsg="32617")
gj.init("/root/nc_spm_full_v2alpha2")

We've launched GRASS GIS now! We can access GRASS GIS commands using the command line interface (with the `!` line magic):

In [ ]:
!g.version

***

## 2. Scripting with GRASS Tools

GRASS functionality is available through tools. There are over 500 different tools in the core distribution and over 300 addon tools or extensions that can be used to prepare and analyze data.

Tools respect the following naming conventions:

Prefix | Function | Example
------ | -------- | -------
r.* | raster processing | r.mapcalc: map algebra
v.*	| vector processing	| v.clean: topological cleaning
i.*	| imagery processing | i.segment: object recognition
db.* | database management | db.select: select values from table
r3.* | 3D raster processing | r3.stats: 3D raster statistics
t.* | temporal data processing | t.rast.aggregate: temporal aggregation
g.* | general data management | g.rename: renames map
d.* | display | d.rast: display raster map

Note also that some tools have multiple dots in their names. For example, tools staring with v.net.* deal with vector network analysis and r.in.* tools import raster data into GRASS GIS spatial database.

Check out the [manual page](https://grass.osgeo.org/grass-devel/manuals/full_index.html) to browse tools.

We'll use the `grass.tools` Python API accessing these tools.

In [ ]:
tools = Tools()

In [ ]:
tools.g_version()

In [ ]:
tools.g_region(flags="p", format="plain")

In [ ]:
tools.g_region(raster="elevation", flags="p")

#### Let's check out our data

In [ ]:
layers = tools.g_list(type="all", format="plain")
print(layers)

`grass.jupyter.Map()` creates and displays GRASS maps as PNG images. Similar to `grass.tools`, `gj.Map()` accepts any GRASS display module as a method by replacing the "." with "_" in the module name. For example:

In [ ]:
example = gj.Map()
example.d_rast(map="elevation")  # d.rast map=elevation
example.d_barscale(bgcolor="none")  # d.barscale
example.d_legend(raster="elevation", flags="b", fontsize="10")  # d.legend
example.show()

To display the image, we call the `show()` method. You can also save the image with the `save()` method.

`grass.jupyter.InteractiveMap()` creates leaflet maps that are interactive. They can also be saved as html with the `save()` method and embedded on a website or shared.

In [ ]:
map = gj.InteractiveMap()
map.add_raster("elevation", opacity=0.6)
map.show()

#### Building a function

We're going to use [r.sim.water](https://grass.osgeo.org/grass-devel/manuals/r.sim.water.html) to model overland flow. The `r.sim.water` tool is the GRASS implementation of the SIMWE model ([Mitas and Mitasova, 1998](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/97WR03347)), a monte carlo path-tracing approach to solving the St. Venant equations for overland flow.

In [ ]:
tools.r_slope_aspect(elevation="elevation", dx="dx", dy="dy")
tools.r_sim_water(
    elevation="elevation",
    dx="dx",
    dy="dy",
    depth="depth"
)

Look at the result (or run other tools to see if it works as expected):

In [ ]:
img = gj.Map()
img.d_rast(map="depth")
img.show()

If that works, let's put it into a Python function. Python function definition starts with `def` followed by a space, name of the function, parentheses which contain parameters, colon, and a function body which is an indented block of code. General syntax looks like this:

```python
def function_name(a, b):
    # code in the function, e.g.,
    return a + b
```

This function takes two parameters, adds them together with `+`, and returns the result.

Our function will have three parameters, namely elevation, rainfall_rate, and _env_. The _env_ parameter is the system environment variables which is optional here but we'll talk more about later when we start writing scripts.

Here is our function which calls _r.sim.water_:

In [ ]:
def run_simwe(elevation, rainfall_rate=40):
    tools.r_slope_aspect(elevation=elevation, dx="dx", dy="dy")
    tools.r_sim_water(
        elevation=elevation,
        dx="dx",
        dy="dy",
        rain=rainfall_rate,
        depth="depth",
        flags="t",
        niterations=30,
    )

Our function creates new raster data, but does not return any value, so in Python, we don't need to have any `return` statement.

Calling a custom function is compltely the same as calling any other function, so:

In [ ]:
run_simwe(elevation="elevation")

Check the result:

In [ ]:
tools.r_info(map="depth.30", flags="g")

In [ ]:
img = gj.Map()
img.d_rast(map="depth.30")
img.show()

In [ ]:
# TODO Add nicer temporal visualization here
# We could have the script write a GIF using gj.Timeseries.save()?

***

## 3. From Function to Executable Script

A basic Python can have a very simple structure:

In [ ]:
%%python
print("Hello!")

Here, the `%%python` line is a IPython kernel magic telling the Jupyter notebook that the cell is a content of a file which should be executed as a Python script. The Python script itself has only one line and that is `print("Hello!")`. In a text editor, you would leave out the notebook-specific  `%%python`, but here we will continue using it so that running the cell actually executes its content as a Python script.

For our non-trivial Python script, we will use a slighly more complex structure which includes `#!` line as a first line, a main function called _main_, and the call of this function in a "if-name-equals-main" block. A full, but minimal, script has these seven lines:

In [ ]:
%%python
#!/usr/bin/env python3


def main():
    pass


if __name__ == "__main__":
    main()

The block which starts with `if __name__ == '__main__'` is what Python will execute when the file is used as a Python script. To keep our code organized, we call there the _main_ function. The _main_ function now contains only `pass` which means tells Python to do nothing. We will replace this `pass` statement with our code. In a script we will create, all code we want to execute will be in the _main_ function, but we will also create other functions and call them from the _main_ function.

Finally, the `#!` line is called a shebang and is used on Linux and macOS and we won't be interacting with it. Note, again, that the `%%python` line is specific to Jupyter and it is not a part of the actual script.

#### GRASS Function to GRASS Script

Now, we add the _run_simwe_ function we have defined earlier, and we add new code to the _main_ function. The complete file is:

In [ ]:
%%python
#!/usr/bin/env python3

import subprocess
import sys


def run_simwe(elevation, rainfall_rate=40, tools=None):
    tools.r_slope_aspect(elevation=elevation, dx="dx", dy="dy")
    tools.r_sim_water(
        elevation=elevation,
        dx="dx",
        dy="dy",
        rain=rainfall_rate,
        depth="depth",
        flags="t",
        niterations=30,
    )


def main():
    # Append GRASS to the python system path
    sys.path.append(
        subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
    )

    import grass.script as gs
    from grass.tools import Tools

    # Initialize the GRASS session
    with gs.setup.init("path/to/my_project") as session:
        tools = Tools(session=session)
        tools.g_region(raster="elevation")
        run_simwe(elevation="elevation", rainfall_rate=60, tools=tools)


if __name__ == "__main__":
    main()

Replace the `%%python` magic at the beginning of the cell with `%%file my_grass_script.py` to write the contents of the cell into a separate file. Now, we can call this script from the terminal!